In [37]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from data_preprocessing import MultiTransformOLIVESDataset  # Assuming the dataset code is in `data_preprocessing.py`
from resnet50_mlp_fusion import SupCEResNet_Fusion  

# Paths
DATA_PATH = '/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES/OLIVES'
CURRENT_DIR = os.getcwd()
LABEL_DATA_PATH = os.path.join(CURRENT_DIR, '..', 'OLIVES_Dataset_Labels', 'full_labels', 'Biomarker_Clinical_Data_Images.csv')

# Biomarker Columns
BIOMARKER_COLUMNS = [
    'Atrophy / thinning of retinal layers', 'Disruption of EZ', 'DRIL', 'IR hemorrhages',
    'IR HRF', 'Partially attached vitreous face', 'Fully attached vitreous face',
    'Preretinal tissue/hemorrhage', 'DRT/ME', 'Fluid (IRF)', 'Fluid (SRF)',
    'Disruption of RPE', 'PED (serous)', 'SHRM'
]

# Transformation Pipelines
TRANSFORMS_LIST = [
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        AddSaltPepperNoise(amount=0.01),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        AddGaussianNoise(mean=0, variance=0.01),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        AddRandomBrightnessContrast(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    transforms.Compose([
        transforms.Resize((224, 224)),
        AddRandomRotation(max_angle=30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
]

# Dataset and DataLoader
dataset = MultiTransformOLIVESDataset(
    csv_path=LABEL_DATA_PATH,
    image_root=DATA_PATH,
    biomarker_cols=BIOMARKER_COLUMNS,
    transforms_list=TRANSFORMS_LIST,
    num_transforms=len(TRANSFORMS_LIST)
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=1)#, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=1)#, pin_memory=True)

# Initialize Model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [32]:
model = SupCEResNet_Fusion(
    name='resnet50',
    num_classes=2,  # Binary classification
    num_features=len(BIOMARKER_COLUMNS),  # Number of biomarker features
    in_channels=3  # RGB input
)
# Dataset and DataLoader
dataset = MultiTransformOLIVESDataset(
    csv_path=LABEL_DATA_PATH,
    image_root=DATA_PATH,
    biomarker_cols=BIOMARKER_COLUMNS,
    transforms_list=TRANSFORMS_LIST,
    num_transforms=len(TRANSFORMS_LIST)
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)


/home/hice1/cphan36/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [39]:
import torch
from train import train, validate
from resnet50_mlp_fusion import SupCEResNet_Fusion  

# Model setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SupCEResNet_Fusion(
    name='resnet50',
    num_classes=1,
    num_features=len(BIOMARKER_COLUMNS)
)

# Loss and optimizer
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc = validate(model, val_loader, criterion)#, DEVICE)

    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [38]:
for i, (images, biomarkers, labels) in enumerate(train_loader):
    if torch.any(labels < 0) or torch.any(labels >= n_classes):
        print(f"Invalid labels in batch {i}: {labels}")
        break


NameError: name 'n_classes' is not defined